In [1]:
import pandas as pd 
import numpy as np 

In [2]:
datos = pd.read_excel("datos/Base_Datos_Presidencia_Cam (1).xlsx",skiprows=12,sheet_name=0)

datos.TIPO_VALORACION=datos.TIPO_VALORACION.map({"SIN VALORACIÓN":"S","POSITIVA":"P","NEGATIVA":"N"})

In [3]:
def tiempoSegundos(x):
    tiempo_separado = x.split(":")
    segundos = 3600 * int(tiempo_separado[0]) + 60 * int(tiempo_separado[1]) + int(tiempo_separado[2])
    return segundos

In [4]:

datos["DURACION_SEGUNDOS"] = datos.DURACION_PIEZA.apply(lambda x : tiempoSegundos(x))

datos["DURACION_MINUTOS"] = datos["DURACION_SEGUNDOS"]/60
datos["Fecha"]=pd.to_datetime(datos.FECHA_TESTIGO, format="%d/%m/%Y")
datos.sort_values(by="Fecha").FECHA_TESTIGO.unique()

In [8]:
datos["FRASE_ESTEREOTIPO_GENERAL"]=datos[["PRESENCIA DE ESTEREOTIPOS","FRASE_ESTEREOTIPO_VULNERABILIDAD"]].apply(lambda x : x.dropna().unique()[0] if len(x.dropna().unique()) else np.nan,axis=1)

In [13]:
datos["FRASE_POSITIVA"] = datos.apply(lambda x :  x.ADJETIVO_FRASE if str(x.TIPO_VALORACION)=="P" else np.nan,axis=1)

datos["FRASE_NEGATIVA"] = datos.apply(lambda x :  x.ADJETIVO_FRASE if str(x.TIPO_VALORACION)=="N" else np.nan,axis=1)

In [14]:
def concatenar_frases(frases):
    frases_no_vacias = list(set(frase for frase in frases if frase.strip()))
    return '... <br/>'.join(frases_no_vacias) if frases_no_vacias else np.nan

In [58]:
datum = datos.sort_values(by="Fecha",ascending=False)[["NOMBRE_NOTICIERO","ID_PIEZA_INFORMATIVA","FECHA_TESTIGO","TIPO_VALORACION","CANDIDATO","DURACION_MINUTOS","FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]]

datos_frases = datum.loc[datum[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]].dropna(how="all").index]
datos_sin_frases = datum.loc[[i for i in  datum.index if i not in datos_frases.index]]

datos_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]] = datos_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]].fillna("")
datos_sin_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]] = datos_sin_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]].fillna("")

datum_frases_agrupadas = datos_frases.groupby(["NOMBRE_NOTICIERO",'ID_PIEZA_INFORMATIVA', 'FECHA_TESTIGO',
       'TIPO_VALORACION', 'CANDIDATO', 'DURACION_MINUTOS']).agg(concatenar_frases).reset_index()

datum_sin_frases_agrupadas=datos_sin_frases.groupby(["NOMBRE_NOTICIERO","ID_PIEZA_INFORMATIVA","FECHA_TESTIGO","TIPO_VALORACION","CANDIDATO","FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]).sum().reset_index()

ladata = pd.concat([datum_frases_agrupadas,datum_sin_frases_agrupadas])

dict_not_id = {}
dict_id_not ={}
a=1
for noticiero in ladata.NOMBRE_NOTICIERO.unique():
    dict_not_id[noticiero] = a
    dict_id_not[a] = noticiero
    a+=1
ladata["ID_NOTICIERO"] = ladata.NOMBRE_NOTICIERO.map(dict_not_id)

ladata.to_csv("datos_limpios/lista_medios.csv",index=False)
pd.DataFrame([dict_id_not]).T[0].to_json("datos_limpios/diccionario_medios.json")
